In [ ]:
#default_exp functional

In [ ]:
#export
from kesscore.imports import *
from functools import wraps

In [ ]:
#export
def _wrapify(f, t):
    @wraps(f)
    def _inner(*args,**kwargs): return t(f(*args,**kwargs))
    return _inner
_listify = partial(_wrapify, t=list)
_Listify = partial(_wrapify, t=L)

In [ ]:
#export
lmap = _listify(map)
lzip = _listify(zip)
lfilter = _listify(filter)
lrange = _listify(range)
Lmap = _Listify(map)
Lzip = _Listify(zip)
Lfilter = _Listify(filter)
Lrange = _Listify(range)

In [ ]:
test_eq_type(lmap(noop, [1,2,3]),          [1,2,3])
test_eq_type(lzip([1,2],[3,4]),            [(1,3),(2,4)])
test_eq_type(lfilter(ge(2), [1,2,3,4]),    [2,3,4])
test_eq_type(lrange(3),                    [0,1,2])
test_eq_type(Lmap(noop, [1,2,3]),          L([1,2,3]))
test_eq_type(Lzip([1,2],[3,4]),            L([(1,3),(2,4)]))
test_eq_type(Lfilter(ge(2), [1,2,3,4]),    L([2,3,4]))
test_eq_type(Lrange(3),                    L([0,1,2]))

In [ ]:
#export
def _all(self:L)->bool: return all(self)
L.all = _all

In [ ]:
assert not L([True, False, True] ).all()
assert not L([True, True,  False]).all()
assert     L([True, True,  True] ).all()

In [ ]:
#export
@patch
def all_eq(self:L)->bool: return all([self[i] == self[i-1] for i in range(1, len(self))])

In [ ]:
assert L(1,1,1,1).all_eq()
assert not L(1,10).all_eq()
assert L(1, True).all_eq()

In [ ]:
#export
@patch
def assert_all_eq(self:L)->bool: assert self.all_eq(), f'Not all elements are equal. {self.unique()=}. ({coll_repr(self)})'

In [ ]:
test_fail(lambda:L(1,2,3).assert_all_eq(), contains='Not all elements are equal. self.unique()=[1, 2, 3]')
import decimal
L(1,True,1.0,decimal.Decimal('1.00000000')).assert_all_eq()

In [ ]:
#export
@patch 
def assert_eq(self:L, other)->bool:
    assert isinstance(other, (L, tuple, list)), f'Unsupported type to compare with {type(other)=}'
    assert len(self) == len(other), f'list lengths are not equal. {len(self)=}, {len(other)=}'
    for i in range(len(self)): assert self[i] == other[i], f'element {i=} is not equal in lists. {self[i]=}, {other[i]=}'

In [ ]:
test_fail(lambda:L(1,2,3).assert_eq([1,2,3,4]), contains='list lengths are not equal. len(self)=3, len(other)=4')
test_fail(lambda:L(1,2,3).assert_eq([1,3,2]),   contains='element i=1 is not equal in lists. self[i]=2, other[i]=3')
L(1,2,3).assert_eq([1,2,3])

In [ ]:
#export
_isinstance = isinstance
def isinstance(obj, class_or_tuple=None):
    'Like isinstance but support creating partial versions, e.g, isinstance(int)(1) is True.'
    if class_or_tuple is None:
        class_or_tuple = obj
        def _inner(obj): 
            return _isinstance(obj, class_or_tuple)
        _inner.__doc__ = f'Check if `obj` is of type\\s: {class_or_tuple}'
        return _inner
    return _isinstance(obj,class_or_tuple)

In [ ]:
assert isinstance(obj=1, class_or_tuple=int)
assert isinstance(int)(1)
test_fail(lambda: isinstance(1)(int), contains='isinstance() arg 2 must be a type or tuple of types')

In [ ]:
assert isinstance(int)(1)
assert all(map(isinstance((list, int)), [1, 2, 3, [1, 2, 3]]))
assert not isinstance((list, int))((1, 2, 3))

In [ ]:
#export
def compose_star(*funcs):
    'like compose but pass *x to functions'
    def _inner(*x):
        for f in funcs: x = f(*x)
        return x
    return _inner

In [ ]:
from nbdev.sync import notebook2script

In [ ]:
notebook2script()

Converted 00_functional.ipynb.
Converted 01_images.ipynb.
Converted 02_download.ipynb.
Converted 03_tensor.ipynb.
Converted 04_random.ipynb.
Converted index.ipynb.
